In [2]:
import pandas as pd
import os

# Paths
base_path = 'C:/Users/uabic/Desktop'
gene_sets_folder = os.path.join(base_path, 'Gene sets')

# Create folder if not exists
if not os.path.exists(gene_sets_folder):
    os.makedirs(gene_sets_folder)

# File pairs and corresponding log-normalized expression data
file_pairs = [
    ('HVGIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('HVGK27M.csv', 'K27Mreal_log_normalized.csv'),
    ('TopFeatIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('TopFeatK27M.csv', 'K27Mreal_log_normalized.csv'),
    ('scEpathIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('scEpathK27M.csv', 'K27Mreal_log_normalized.csv')
]

# Process each pair
for gene_file, expression_file in file_pairs:
    gene_file_path = os.path.join(base_path, gene_file)
    expression_file_path = os.path.join(base_path, expression_file)

    # Load gene names
    genes_df = pd.read_csv(gene_file_path)
    gene_names = genes_df['Gene'].tolist()

    # Load expression data
    expression_df = pd.read_csv(expression_file_path)
    selected_expression_df = expression_df[expression_df['Gene'].isin(gene_names)]

    # Save selected expression data
    output_file_path = os.path.join(gene_sets_folder, f'selected_{gene_file}')
    selected_expression_df.to_csv(output_file_path, index=False)
    
#Julia is used for the Network inference with PID: infer the network using the NetworkInference.jl package for each selected gene sets

In [2]:
import pandas as pd
import os

# Paths
base_path = 'C:/Users/uabic/Desktop'
gene_sets_folder = os.path.join(base_path, 'Gene sets')

# Convert CSV to TXT
def convert_csv_to_txt(csv_file_path, txt_file_path):
    df = pd.read_csv(csv_file_path)
    df.to_csv(txt_file_path, sep='\t', index=False, header=False)

# File pairs and corresponding log-normalized expression data
file_pairs = [
    ('HVGIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('HVGK27M.csv', 'K27Mreal_log_normalized.csv'),
    ('TopFeatIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('TopFeatK27M.csv', 'K27Mreal_log_normalized.csv'),
    ('scEpathIDHWT.csv', 'IDHWT_log_normalized.csv'),
    ('scEpathK27M.csv', 'K27Mreal_log_normalized.csv')
]

# Convert each selected gene set CSV to TXT
for gene_file, _ in file_pairs:
    csv_file_path = os.path.join(gene_sets_folder, f'selected_{gene_file}')
    txt_file_path = os.path.join(gene_sets_folder, f'selected_{gene_file.replace(".csv", ".txt")}')
    convert_csv_to_txt(csv_file_path, txt_file_path)
    
    
#Proceed with the NetworkInference.jl package code from Julia to infer the PID networks.

In [8]:
#Network Plotting and Centrality Measures

import pandas as pd
import os
import networkx as nx
import matplotlib.pyplot as plt

# Define colors for different datasets
color_mapping = {
    'HVGIDHWT': 'violet',
    'HVGK27M': 'violet',
    'TopFeatIDHWT': 'skyblue',
    'TopFeatK27M': 'skyblue',
    'scEpathIDHWT': 'turquoise',
    'scEpathK27M': 'turquoise'
}

# Plot and save network function
def plot_network(G, color, output_file):
    pos = nx.spring_layout(G)
    plt.figure(figsize=(18, 18), dpi=150)
    nx.draw(G, pos, node_color=color, edge_color='pink', with_labels=True, labels={node: node for node in G.nodes})
    plt.savefig(output_file, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Calculate and save centrality measures
def calculate_centrality_measures(G, output_folder):
    centrality_measures = {
        'betweenness': nx.betweenness_centrality(G),
        'closeness': nx.closeness_centrality(G),
        'eigenvector': nx.eigenvector_centrality(G)
    }

    for measure, values in centrality_measures.items():
        output_file_path = os.path.join(output_folder, f'{measure}_centrality.csv')
        pd.DataFrame.from_dict(values, orient='index', columns=[measure]).to_csv(output_file_path)

    return centrality_measures

# Process each gene set
base_path = 'C:/Users/uabic/Desktop'
gene_sets_folder = os.path.join(base_path, 'Gene sets')

file_names = ["PID_selected_HVGIDHWT", "PID_selected_HVGK27M", "PID_selected_TopFeatIDHWT", "PID_selected_TopFeatK27M", "PID_selected_scEpathIDHWT", "PID_selected_scEpathK27M"]

for file_name in file_names:
    network_file_path = os.path.join(gene_sets_folder, f'{file_name}.txt')
    
    # Load network data
    network_df = pd.read_csv(network_file_path, sep='\t', header=None, names=['source', 'target', 'weight'])
    G = nx.from_pandas_edgelist(network_df, source='source', target='target', edge_attr='weight')
    
    # Calculate centrality measures
    output_folder = os.path.join(gene_sets_folder, file_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    centrality_measures = calculate_centrality_measures(G, output_folder)
    
    # Plot and save network
    output_image_path = os.path.join(gene_sets_folder, f'network_{file_name}.jpeg')
    plot_network(G, color_mapping[file_name.split('_')[2]], output_image_path)


In [12]:
#BDM Single Genes

import pandas as pd

# Paths
base_path = 'C:/Users/uabic/Desktop/Gene sets'
file_names = ["selected_scEpathIDHWT.csv", "selected_scEpathK27M.csv"]

# Load and binarize data
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    data_df = pd.read_csv(file_path, index_col='Gene')
    binarized_data = (data_df > 0.5).astype(int)
    print(f'Binarized data for {file_name}:\n', binarized_data.head())
    
from pybdm import BDM

bdm = BDM(ndim=1)

# Function to compute BDM for each gene
def compute_bdm_for_genes(data):
    bdm_values = []
    for gene in data.index:
        gene_data = data.loc[gene].values
        bdm_value = bdm.bdm(gene_data)
        bdm_values.append((gene, bdm_value))
    return bdm_values

# Compute BDM for binarized data
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    data_df = pd.read_csv(file_path, index_col='Gene')
    binarized_data = (data_df > 0.5).astype(int)
    bdm_values = compute_bdm_for_genes(binarized_data)
    bdm_df = pd.DataFrame(bdm_values, columns=['Gene', 'BDM'])
    print(f'BDM values for {file_name}:\n', bdm_df.head())



Binarized data for selected_scEpathIDHWT.csv:
         BT749-P11-A01  BT749-P11-A02  BT749-P11-A03  BT749-P11-A04  \
Gene                                                                 
ARID5B              1              0              0              1   
DLX1                0              0              0              0   
HMGB3               0              0              0              0   
KDM5B               0              1              0              0   
SOX11               1              1              1              1   

        BT749-P11-A05  BT749-P11-A07  BT749-P11-A08  BT749-P11-A09  \
Gene                                                                 
ARID5B              1              0              1              0   
DLX1                0              0              0              0   
HMGB3               0              1              1              1   
KDM5B               1              0              0              1   
SOX11               1              1      

In [13]:
import os

# Output folder
output_folder = os.path.join(base_path, 'SingleBDM')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save BDM values to CSV
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    data_df = pd.read_csv(file_path, index_col='Gene')
    binarized_data = (data_df > 0.5).astype(int)
    bdm_values = compute_bdm_for_genes(binarized_data)
    bdm_df = pd.DataFrame(bdm_values, columns=['Gene', 'BDM'])
    bdm_csv_path = os.path.join(output_folder, f'BDM_values_{file_name.split(".")[0]}.csv')
    bdm_df.to_csv(bdm_csv_path, index=False)
    print(f'Saved BDM values to {bdm_csv_path}')
import matplotlib.pyplot as plt
import numpy as np

# Colors for plots
colors = {'selected_scEpathIDHWT': 'violet', 'selected_scEpathK27M': 'pink'}

# Plot top 10 genes with highest BDM
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    data_df = pd.read_csv(file_path, index_col='Gene')
    binarized_data = (data_df > 0.5).astype(int)
    bdm_values = compute_bdm_for_genes(binarized_data)
    bdm_df = pd.DataFrame(bdm_values, columns=['Gene', 'BDM'])
    bdm_df.sort_values(by='BDM', ascending=False, inplace=True)
    top_bdm_df = bdm_df.head(10)
    
    # Compute standard error
    std_error = np.std(top_bdm_df['BDM']) / np.sqrt(len(top_bdm_df))
    
    # Plot bar graph
    plt.figure(figsize=(12, 8), dpi=150)
    plt.bar(top_bdm_df['Gene'], top_bdm_df['BDM'], color=colors[file_name.split('.')[0]], yerr=std_error, capsize=5)
    plt.xlabel('Genes')
    plt.ylabel('BDM (bits)')
    plt.title('Estimated Algorithmic Complexity of Transition Genes')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Save plot
    plot_file_path = os.path.join(output_folder, f'top10_bdm_{file_name.split("_")[1]}.jpeg')
    plt.savefig(plot_file_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()
    print(f'Saved plot to {plot_file_path}')


Saved BDM values to C:/Users/uabic/Desktop/Gene sets\SingleBDM\BDM_values_selected_scEpathIDHWT.csv
Saved BDM values to C:/Users/uabic/Desktop/Gene sets\SingleBDM\BDM_values_selected_scEpathK27M.csv
Saved plot to C:/Users/uabic/Desktop/Gene sets\SingleBDM\top10_bdm_scEpathIDHWT.csv.jpeg
Saved plot to C:/Users/uabic/Desktop/Gene sets\SingleBDM\top10_bdm_scEpathK27M.csv.jpeg


In [ ]:
from pybdm import BDM
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

bdm = BDM(ndim=2)

# Function to calculate BDM perturbation
def bdm_perturbation(data):
    original_bdm = bdm.bdm(data)
    perturbation_results = []

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            perturbed_data = data.copy()
            perturbed_data[i, j] = 1 - perturbed_data[i, j]  # Flip bit
            perturbed_bdm = bdm.bdm(perturbed_data)
            perturbation_results.append(((i, j), abs(perturbed_bdm - original_bdm)))

    perturbation_results.sort(key=lambda x: x[1], reverse=True)
    return perturbation_results

# Binarize adjacency matrix
def binarize_matrix(matrix, threshold=0.5):
    binarized_matrix = (matrix > threshold).astype(int)
    return binarized_matrix

# Process each gene set
base_path = 'C:/Users/uabic/Desktop'
gene_sets_folder = os.path.join(base_path, 'Gene sets')

file_names = ["PID_selected_HVGIDHWT", "PID_selected_HVGK27M", "PID_selected_TopFeatIDHWT", "PID_selected_TopFeatK27M"]

for file_name in file_names:
    adjacency_file_path = os.path.join(gene_sets_folder, f'{file_name}.txt')

    # Load adjacency matrix
    adjacency_matrix = pd.read_csv(adjacency_file_path, sep='\t', header=None).pivot_table(index=0, columns=1, values=2).fillna(0).values
    
    # Binarize adjacency matrix
    binarized_matrix = binarize_matrix(adjacency_matrix)
    
    # Perform BDM perturbation analysis
    bdm_results = bdm_perturbation(binarized_matrix)

    # Save BDM perturbation results
    output_folder = os.path.join(gene_sets_folder, file_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    bdm_results_file_path = os.path.join(output_folder, 'bdm_perturbation.csv')
    pd.DataFrame(bdm_results, columns=['Interaction', 'BDM Change']).to_csv(bdm_results_file_path, index=False)

    # Plot top 10 BDM perturbations
    top_bdm_results = bdm_results[:10]
    interactions, changes = zip(*top_bdm_results)
    interactions = [f'{i[0]}-{i[1]}' for i in interactions]

    plt.figure(figsize=(10, 6))
    plt.bar(interactions, changes, color='purple')
    plt.xlabel('Interaction')
    plt.ylabel('BDM Change')
    plt.title('Top 10 BDM Perturbations')
    plt.xticks(rotation=45)
    plt.savefig(os.path.join(output_folder, 'top_10_bdm_perturbations.jpeg'), format='jpeg', pil_kwargs={'quality': 95})
    plt.close()



In [7]:
import pandas as pd
import numpy as np
from pybdm import BDM
from pybdm import PerturbationExperiment
import matplotlib.pyplot as plt
import networkx as nx
import os

# Set the working directory
os.chdir(r'C:\Users\uabic\Desktop')

# Define the filenames
file_names = ["PID_selected_HVGIDHWT.txt", "PID_selected_HVGK27M.txt", "PID_selected_TopFeatIDHWT.txt", 
              "PID_selected_TopFeatK27M.txt", "PID_selected_scEpathIDHWT.txt", "PID_selected_scEpathK27M.txt"]

# Create a directory for saving results
output_dir = 'Networkx'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform BDM perturbation analysis
def binarize_matrix(matrix, threshold=0.5):
    binary_matrix = (matrix > threshold).astype(int)
    return binary_matrix

def bdm_perturbation_analysis(adj_matrix):
    bdm = BDM(ndim=2)
    binary_matrix = binarize_matrix(adj_matrix.values)
    perturbation = PerturbationExperiment(bdm, binary_matrix, metric='bdm')
    delta_bdm = perturbation.run()
    
    # Ensure the shape matches the original matrix
    reshaped_delta_bdm = np.reshape(delta_bdm, adj_matrix.shape)
    
    return pd.DataFrame(reshaped_delta_bdm, index=adj_matrix.index, columns=adj_matrix.columns)

# Function to get top 5 unique gene pairs
def get_top_5_unique_pairs(bdm_results):
    bdm_unstacked = bdm_results.unstack()
    sorted_bdm = bdm_unstacked.sort_values(ascending=False)
    unique_pairs = {}
    
    for (gene1, gene2), value in sorted_bdm.items():
        sorted_pair = tuple(sorted([gene1, gene2]))
        if sorted_pair not in unique_pairs:
            unique_pairs[sorted_pair] = value
        if len(unique_pairs) == 5:
            break
    
    top_5_pairs = pd.Series(unique_pairs).sort_values(ascending=False)
    return top_5_pairs

# Function to plot the top 5 BDM changes
def plot_top_5_bdm_changes(bdm_results, title, save_path, color):
    top_5 = get_top_5_unique_pairs(bdm_results)
    means = top_5.values
    errors = np.std(top_5.values) / np.sqrt(len(top_5.values))
    
    plt.figure(figsize=(20, 20))
    plt.bar(top_5.index.map(str), means, yerr=errors, color=[color] * len(top_5), capsize=5)
    plt.ylabel('BDM Change (bits)', fontsize=18)
    plt.xlabel('Gene-Gene Interaction', fontsize=18)
    plt.title(title, fontsize=18)
    plt.xticks(rotation=45, ha='right', fontsize=18)
    plt.yticks(fontsize=18)
    
    plt.savefig(save_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Function to plot the network
def plot_network(adj_matrix, title, save_path):
    G = nx.from_pandas_adjacency(adj_matrix)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(24, 16))
    nx.draw(G, pos, with_labels=False, node_color='skyblue', edge_color='pink', node_size=500)
    
    # Move the labels to avoid overlap
    pos_labels = {node: (x, y + 0.1) for (node, (x, y)) in pos.items()}
    nx.draw_networkx_labels(G, pos_labels, font_size=24, font_color='black')
    
    plt.title(title, fontsize=24)
    plt.tight_layout()
    plt.savefig(save_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Function to compute centralities and save as CSV
def compute_centralities(adj_matrix, prefix):
    G = nx.from_pandas_adjacency(adj_matrix)
    centralities = {
        'betweenness': nx.betweenness_centrality(G),
        'closeness': nx.closeness_centrality(G),
        'eigenvector': nx.eigenvector_centrality(G, max_iter=1000)
    }
    centrality_df = pd.DataFrame(centralities)
    centrality_df.to_csv(os.path.join(output_dir, f"{prefix}_centrality_measures.csv"))

# Process each file
for file_name in file_names:
    # Load the edge list dataset and assign column names
    pid_data = pd.read_csv(file_name, sep='\t', header=None, names=['from', 'to', 'weight'])
    
    # Convert the edge list to an adjacency matrix
    G = nx.from_pandas_edgelist(pid_data, 'from', 'to', edge_attr='weight')
    adj_matrix = nx.to_pandas_adjacency(G, nodelist=sorted(G.nodes()))
    
    # Save the adjacency matrix
    adj_matrix.to_csv(os.path.join(output_dir, f"{file_name}_adjacency_matrix.csv"))
    
    # Perform BDM perturbation analysis
    bdm_results = bdm_perturbation_analysis(adj_matrix)
    
    # Save BDM results
    bdm_results.to_csv(os.path.join(output_dir, f"{file_name}_bdm_results.csv"))
    
    # Plot and save the top 5 BDM changes
    plot_color = 'lightblue' if 'IDHWT' in file_name else 'lightseagreen'
    plot_top_5_bdm_changes(bdm_results, f'Top 5 BDM Changes for {file_name}', os.path.join(output_dir, f"{file_name}_top_5_bdm_changes.jpeg"), plot_color)
    
    # Plot the network
    plot_network(adj_matrix.abs(), f'Network for {file_name}', os.path.join(output_dir, f"{file_name}_network.jpeg"))
    
    # Compute and save centrality measures
    compute_centralities(adj_matrix.abs(), file_name)


C:\Users\uabic\AppData\Local\Temp/ipykernel_7604/1988522171.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\uabic\AppData\Local\Temp/ipykernel_7604/1988522171.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\uabic\AppData\Local\Temp/ipykernel_7604/1988522171.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\uabic\AppData\Local\Temp/ipykernel_7604/1988522171.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\uabic\AppData\Local\Temp/ipykernel_7604/1988522171.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layo

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os
import numpy as np

# Set the working directory
os.chdir(r'C:\Users\uabic\Desktop')

# Define the filename
file_name = "PID_selected_HVGIDHWT.txt"

# Create a directory for saving results
output_dir = 'Networkx'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to plot the network
def plot_network(adj_matrix, title, save_path, spread_nodes):
    G = nx.from_pandas_adjacency(adj_matrix)
    pos = nx.spring_layout(G)
    fig, ax = plt.subplots(figsize=(24, 16))
    
    # Adjust positions to ensure nodes are 1 cm apart
    for node in pos:
        x, y = pos[node]
        pos[node] = (x + np.random.uniform(-0.1, 0.1), y + np.random.uniform(-0.1, 0.1))
    
    nx.draw(G, pos, with_labels=False, node_color='skyblue', edge_color='pink', node_size=500, ax=ax)
    
    # Circular arrangement for spread nodes
    angle_offset = 2 * np.pi / len(spread_nodes)
    for i, node in enumerate(spread_nodes):
        x, y = pos[node]
        radius = 0.1 if i < len(spread_nodes) / 2 else 0.2
        angle = i * angle_offset
        pos[node] = (x + radius * np.cos(angle), y + radius * np.sin(angle))
    
    nx.draw_networkx_labels(G, pos, font_size=24, font_color='black', ax=ax)
    
    plt.title(title, fontsize=24)
    fig.tight_layout()
    plt.savefig(save_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Load the edge list dataset
pid_data = pd.read_csv(file_name, sep='\t', header=None, names=['from', 'to', 'weight'])

# Convert the edge list to an adjacency matrix
pid_data['from'] = pid_data['from'].astype(str)
pid_data['to'] = pid_data['to'].astype(str)
G = nx.from_pandas_edgelist(pid_data, 'from', 'to', edge_attr='weight')
adj_matrix = nx.to_pandas_adjacency(G, nodelist=sorted(G.nodes(), key=str))

# Plot the network
spread_nodes = ['C1QA', 'C1QB', 'CD14', 'LAPTM5', 'TYROBP', 'VSIG4']
plot_network(adj_matrix.abs(), 'Network for PID_selected_HVGIDHWT', os.path.join(output_dir, 'PID_selected_HVGIDHWT_network.jpeg'), spread_nodes)


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os
import numpy as np

# Set the working directory
os.chdir(r'C:\Users\uabic\Desktop')

# Define the filename
file_name = "PID_selected_HVGIDHWT.txt"

# Create a directory for saving results
output_dir = 'Networkx'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to plot the network
def plot_network(adj_matrix, title, save_path, spread_nodes):
    G = nx.from_pandas_adjacency(adj_matrix)
    pos = nx.spring_layout(G)
    fig, ax = plt.subplots(figsize=(24, 16))
    
    # Adjust positions to ensure nodes are spread apart
    for node in pos:
        x, y = pos[node]
        pos[node] = (x + np.random.uniform(-0.2, 0.2), y + np.random.uniform(-0.2, 0.2))
    
    nx.draw(G, pos, with_labels=False, node_color='skyblue', edge_color='pink', node_size=500, ax=ax)
    
    # Circular arrangement for spread nodes with alternating distances
    angle_offset = 2 * np.pi / len(spread_nodes)
    distances = [0.1, 0.2, 0.3]  # Alternating distances
    for i, node in enumerate(spread_nodes):
        x, y = pos[node]
        radius = distances[i % len(distances)]
        angle = i * angle_offset
        pos[node] = (x + radius * np.cos(angle), y + radius * np.sin(angle))
    
    # Ensure no nodes overlap by adjusting positions further if necessary
    def ensure_no_overlap(pos, spread_nodes, min_distance=0.05):
        adjusted_pos = pos.copy()
        for node in spread_nodes:
            for other_node in spread_nodes:
                if node != other_node:
                    dx = adjusted_pos[node][0] - adjusted_pos[other_node][0]
                    dy = adjusted_pos[node][1] - adjusted_pos[other_node][1]
                    distance = np.sqrt(dx**2 + dy**2)
                    if distance < min_distance:
                        angle = np.arctan2(dy, dx)
                        adjusted_pos[node] = (adjusted_pos[node][0] + min_distance * np.cos(angle),
                                              adjusted_pos[node][1] + min_distance * np.sin(angle))
        return adjusted_pos

    pos = ensure_no_overlap(pos, spread_nodes)
    
    nx.draw_networkx_labels(G, pos, font_size=24, font_color='black', ax=ax)
    
    plt.title(title, fontsize=24)
    fig.tight_layout()
    plt.savefig(save_path, format='jpeg', pil_kwargs={'quality': 95})
    plt.close()

# Load the edge list dataset
pid_data = pd.read_csv(file_name, sep='\t', header=None, names=['from', 'to', 'weight'])

# Convert the edge list to an adjacency matrix
pid_data['from'] = pid_data['from'].astype(str)
pid_data['to'] = pid_data['to'].astype(str)
G = nx.from_pandas_edgelist(pid_data, 'from', 'to', edge_attr='weight')
adj_matrix = nx.to_pandas_adjacency(G, nodelist=sorted(G.nodes(), key=str))

# Plot the network
spread_nodes = ['C1QA', 'C1QB', 'CD14', 'LAPTM5', 'TYROBP', 'VSIG4']
plot_network(adj_matrix.abs(), 'Network for PID_selected_HVGIDHWT', os.path.join(output_dir, 'PID_selected_HVGIDHWT_network.jpeg'), spread_nodes)
